In [53]:
from libs.configuration import configure
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

from shared.spark_config import create_spark_config

env = configure()
conf = create_spark_config("M2_Processors.aircraft.tier1")

In [2]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [98]:
df = spark.read.format("iceberg").load("dev.raw.aircrafts")
df.show()

+--------------------+----+--------------+--------+--------------------+---------+------+------+----+-------------+-------------+----------------+-----------------+----------------+----------------+---------------+-------------+-------------+------------------+
|          created_ts|icao|classification|category|       manufacturers|wing_span|length|height|mtow|fuel_capacity|maximum_range|persons_on_board|take_off_distance|landing_distance|absolute_ceiling|optimum_ceiling|maximum_speed|optimum_speed|maximum_climb_rate|
+--------------------+----+--------------+--------+--------------------+---------+------+------+----+-------------+-------------+----------------+-----------------+----------------+----------------+---------------+-------------+-------------+------------------+
|2024-01-10 14:10:...|A002|           G1P|     L/G|       [IRKUT A-002]|        -|   5.9|   3.3| 0.9|             |          270|               3|               90|              10|             100|             80|

In [99]:
df.schema

StructType([StructField('created_ts', TimestampType(), True), StructField('icao', StringType(), True), StructField('classification', StringType(), True), StructField('category', StringType(), True), StructField('manufacturers', ArrayType(StringType(), True), True), StructField('wing_span', StringType(), True), StructField('length', StringType(), True), StructField('height', StringType(), True), StructField('mtow', StringType(), True), StructField('fuel_capacity', StringType(), True), StructField('maximum_range', StringType(), True), StructField('persons_on_board', StringType(), True), StructField('take_off_distance', StringType(), True), StructField('landing_distance', StringType(), True), StructField('absolute_ceiling', StringType(), True), StructField('optimum_ceiling', StringType(), True), StructField('maximum_speed', StringType(), True), StructField('optimum_speed', StringType(), True), StructField('maximum_climb_rate', StringType(), True)])

In [100]:
df.describe().show()

+-------+----+--------------+--------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|summary|icao|classification|category|         wing_span|            length|            height|              mtow|     fuel_capacity|     maximum_range| persons_on_board|take_off_distance| landing_distance|  absolute_ceiling|   optimum_ceiling|     maximum_speed|     optimum_speed|maximum_climb_rate|
+-------+----+--------------+--------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|  count|2709|          2709|    2709|              2709|              2709|              2709

In [101]:
for c in df.columns:
    if c in ["created_ts", "icao", "classification", "category", "manufacturers"]:
        continue

    df.select(c).where(F.col(c).rlike(r"[^\d\.]")).describe().show()

+-------+---------+
|summary|wing_span|
+-------+---------+
|  count|      228|
|   mean|     NULL|
| stddev|     NULL|
|    min|        -|
|    max|        -|
+-------+---------+

+-------+------+
|summary|length|
+-------+------+
|  count|     7|
|   mean|  NULL|
| stddev|  NULL|
|    min|     -|
|    max|     -|
+-------+------+

+-------+------+
|summary|height|
+-------+------+
|  count|     7|
|   mean|  NULL|
| stddev|  NULL|
|    min|     -|
|    max|     -|
+-------+------+

+-------+----+
|summary|mtow|
+-------+----+
|  count|   7|
|   mean|NULL|
| stddev|NULL|
|    min|   -|
|    max|   -|
+-------+----+

+-------+-------------+
|summary|fuel_capacity|
+-------+-------------+
|  count|           29|
|   mean|         NULL|
| stddev|         NULL|
|    min|            -|
|    max|            -|
+-------+-------------+

+-------+-------------+
|summary|maximum_range|
+-------+-------------+
|  count|            7|
|   mean|         NULL|
| stddev|         NULL|
|    min|     

In [102]:
df = df.replace(
    to_replace={"-": None, "": None},
    subset=[
        "wing_span",
        "length",
        "height",
        "mtow",
        "fuel_capacity",
        "maximum_range",
        "take_off_distance",
        "landing_distance",
        "absolute_ceiling",
        "optimum_ceiling",
        "maximum_climb_rate",
    ],
)

In [103]:
df.select("icao").where(F.col("icao").rlike(r"[^a-zA-Z0-9]")).describe().show()
df.select("classification").where(F.col("classification").rlike(r"[^a-zA-Z0-9]")).describe().show()
df.select("category").where(F.col("category").rlike(r"[^a-zA-Z0-9\/]")).describe().show()

+-------+----+
|summary|icao|
+-------+----+
|  count|   0|
|   mean|NULL|
| stddev|NULL|
|    min|NULL|
|    max|NULL|
+-------+----+

+-------+--------------+
|summary|classification|
+-------+--------------+
|  count|            11|
|   mean|          NULL|
| stddev|          NULL|
|    min|             -|
|    max|           L/G|
+-------+--------------+

+-------+--------+
|summary|category|
+-------+--------+
|  count|      14|
|   mean|    NULL|
| stddev|    NULL|
|    min|      -/|
|    max|   L-M/G|
+-------+--------+



In [104]:
df = df.replace(to_replace={"-": None, "": None}, subset=["classification"])

In [105]:
df = df.replace(to_replace={"-/": None, "-/-": None, "": None}, subset=["category"])

In [106]:
df = (
    df.withColumn("capacity_caps", F.split("persons_on_board", "-"))
    .withColumn("capacity_lower_cap", F.try_element_at("capacity_caps", F.lit(-2)))
    .withColumn("capacity_upper_cap", F.element_at("capacity_caps", F.lit(-1)))
    .drop("persons_on_board", "capacity_caps")
)

df.show()

+--------------------+----+--------------+--------+--------------------+---------+------+------+----+-------------+-------------+-----------------+----------------+----------------+---------------+-------------+-------------+------------------+------------------+------------------+
|          created_ts|icao|classification|category|       manufacturers|wing_span|length|height|mtow|fuel_capacity|maximum_range|take_off_distance|landing_distance|absolute_ceiling|optimum_ceiling|maximum_speed|optimum_speed|maximum_climb_rate|capacity_lower_cap|capacity_upper_cap|
+--------------------+----+--------------+--------+--------------------+---------+------+------+----+-------------+-------------+-----------------+----------------+----------------+---------------+-------------+-------------+------------------+------------------+------------------+
|2024-01-10 14:10:...|A002|           G1P|     L/G|       [IRKUT A-002]|     NULL|   5.9|   3.3| 0.9|         NULL|          270|               90|    

In [107]:
df = (
    df.withColumn(
        "capacity_upper_capx",
        F.split("capacity_upper_cap", r"\+").cast("array<int>"),
    )
    .withColumn(
        "capacity_upper_cap", F.aggregate("capacity_upper_capx", F.lit(0), lambda acc, x: acc + x)
    )
    .drop("capacity_upper_capx")
)

In [108]:
df = df.withColumns(
    {
        "wing_span": F.col("wing_span").cast(T.FloatType()),
        "length": F.col("length").cast(T.FloatType()),
        "height": F.col("height").cast(T.FloatType()),
        "mtow": F.col("mtow").cast(T.FloatType()),
        "fuel_capacity": F.col("fuel_capacity").cast(T.FloatType()),
        "maximum_range": F.col("maximum_range").cast(T.IntegerType()),
        "take_off_distance": F.col("take_off_distance").cast(T.FloatType()),
        "landing_distance": F.col("landing_distance").cast(T.FloatType()),
        "absolute_ceiling": F.col("absolute_ceiling").cast(T.FloatType()),
        "optimum_ceiling": F.col("optimum_ceiling").cast(T.FloatType()),
        "maximum_climb_rate": F.col("maximum_climb_rate").cast(T.FloatType()),
        "capacity_lower_cap": F.col("capacity_lower_cap").cast(T.IntegerType()),
    }
).drop("maximum_speed", "optimum_speed")

In [109]:
df.schema

StructType([StructField('created_ts', TimestampType(), True), StructField('icao', StringType(), True), StructField('classification', StringType(), True), StructField('category', StringType(), True), StructField('manufacturers', ArrayType(StringType(), True), True), StructField('wing_span', FloatType(), True), StructField('length', FloatType(), True), StructField('height', FloatType(), True), StructField('mtow', FloatType(), True), StructField('fuel_capacity', FloatType(), True), StructField('maximum_range', IntegerType(), True), StructField('take_off_distance', FloatType(), True), StructField('landing_distance', FloatType(), True), StructField('absolute_ceiling', FloatType(), True), StructField('optimum_ceiling', FloatType(), True), StructField('maximum_climb_rate', FloatType(), True), StructField('capacity_lower_cap', IntegerType(), True), StructField('capacity_upper_cap', IntegerType(), True)])

In [110]:
df.describe().show()

+-------+----+--------------+--------+------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|summary|icao|classification|category|         wing_span|            length|           height|              mtow|     fuel_capacity|     maximum_range|take_off_distance| landing_distance|  absolute_ceiling|   optimum_ceiling|maximum_climb_rate|capacity_lower_cap|capacity_upper_cap|
+-------+----+--------------+--------+------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|  count|2709|          2701|    2701|              2432|              2586|             2330|              2494|              1852|              2180|

In [111]:
df = df.withColumn("created_ts", F.current_timestamp()).withColumn(
    "updated_ts", F.current_timestamp()
)
df.show()

+--------------------+----+--------------+--------+--------------------+---------+------+------+-----+-------------+-------------+-----------------+----------------+----------------+---------------+------------------+------------------+------------------+--------------------+
|          created_ts|icao|classification|category|       manufacturers|wing_span|length|height| mtow|fuel_capacity|maximum_range|take_off_distance|landing_distance|absolute_ceiling|optimum_ceiling|maximum_climb_rate|capacity_lower_cap|capacity_upper_cap|          updated_ts|
+--------------------+----+--------------+--------+--------------------+---------+------+------+-----+-------------+-------------+-----------------+----------------+----------------+---------------+------------------+------------------+------------------+--------------------+
|2024-01-10 21:42:...|A002|           G1P|     L/G|       [IRKUT A-002]|     NULL|   5.9|   3.3|  0.9|         NULL|          270|             90.0|            10.0|    

In [115]:
df.writeTo("dev.tier1.aircrafts").append()